In [1]:
import warnings
warnings.filterwarnings('ignore')

import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import sklearn.preprocessing
from sklearn.metrics import r2_score

from keras.layers import Dense,Dropout,SimpleRNN,LSTM
from keras.models import Sequential

plt.rcParams['font.family'] = 'Malgun Gothic'

# 데이터 로드

In [2]:
df = pd.read_csv('./train.csv')
df['시점'] = pd.to_datetime(df['시점'], format='%Y-%m-%d %H')
df.set_index('시점', inplace=True)
df

,구분,공급량
시점,,
2013-01-01 00:00:00,A,2497.129
2013-01-01 01:00:00,A,2363.265
2013-01-01 02:00:00,A,2258.505
2013-01-01 03:00:00,A,2243.969
2013-01-01 04:00:00,A,2344.105
...,...,...
2018-12-31 19:00:00,H,681.033
2018-12-31 20:00:00,H,669.961
2018-12-31 21:00:00,H,657.941


In [3]:
test = pd.read_csv('./test.csv')
test['시점'] = pd.to_datetime(test['시점'], format='%Y-%m-%d %H')
test.set_index('시점', inplace=True)
test

,구분
시점,
2019-01-01 00:00:00,A
2019-01-01 01:00:00,A
2019-01-01 02:00:00,A
2019-01-01 03:00:00,A
2019-01-01 04:00:00,A
...,...
2019-03-31 19:00:00,H
2019-03-31 20:00:00,H
2019-03-31 21:00:00,H


# 함수 및 상수 선언

In [4]:
def load_data(data, seq_len):
    X = []
    y = []
    for i in range(seq_len, len(data)):
        X.append(data.iloc[i-seq_len : i, 0])
        y.append(data.iloc[i, 0])
    
    X = np.array(X)
    y = np.array(y)
        
    X = np.reshape(X, (X.shape[0], seq_len, 1))
            
    return [X, y]

In [5]:
def loss_nmae(true, pred):
    score = tf.reduce_mean((tf.math.abs(true-pred))/true)
    return score

In [6]:
def build_model(X, learning_rate):
    lstm_model = Sequential()

    lstm_model.add(LSTM(40,activation="tanh",return_sequences=True, input_shape=(X.shape[1],1)))

    lstm_model.add(LSTM(40,activation="tanh",return_sequences=True))

    lstm_model.add(LSTM(40,activation="tanh",return_sequences=False))

    lstm_model.add(Dense(1))
    
    adam = tf.keras.optimizers.Adam(lr=learning_rate)

    lstm_model.compile(optimizer=adam, loss=loss_nmae)
    
    return lstm_model

In [7]:
def grid_search(data, hyp_parameter):
    seq_len, learning_rate, epochs, batch_size = hyp_parameter
    val_loss_mean = 0
    X, y = load_data(data, seq_len)
    print('------------------------------------------------------')
    print('Hyper Parameter: seq_len=%d, learning_rate=%.2f, epochs=%d, batch_size=%d' %(seq_len, learning_rate, epochs, batch_size))
    for i in range(10):
        lstm_model = build_model(X, learning_rate)
        grid = lstm_model.fit(X, y, epochs=epochs, batch_size=batch_size, validation_split=0.2, verbose=0)
        val_loss = grid.history['val_loss'][-1]
        print('> %d) nmae: %.5f' %(i+1, val_loss))
        val_loss_mean += val_loss
    val_loss_mean = val_loss_mean / 10
    print('>> nmae mean: %.5f' %val_loss_mean)
    return [seq_len, learning_rate, epochs, batch_size, val_loss_mean]

In [8]:
def search_parameter(data, hyp_parameters):
    NMAE = pd.DataFrame(columns=['seq_len', 'learning_rate', 'epochs', 'batch_size', 'nmae'])
    for hyp_parameter in hyp_parameters:
        result = grid_search(data, hyp_parameter)
        NMAE.loc[len(NMAE)] = result
    NMAE = NMAE.sort_values('nmae')
    seq_len = NMAE['seq_len'].iloc[0]
    learning_rate = NMAE['learning_rate'].iloc[0]
    epochs = NMAE['epochs'].iloc[0]
    batch_size = NMAE['batch_size'].iloc[0]
    nmae = NMAE['nmae'].iloc[0]
    print('-----------------------------------------------------------------------------------')
    print('최적 hyper parameter: seq_len=%d, learning_rate=%.2f, epohcs= %d, batch_size=%d -> nmae=%.5f' 
          %(seq_len, learning_rate, epochs, batch_size, nmae))
    return int(seq_len), learning_rate, int(epochs), int(batch_size)

In [9]:
def set_hyp_parameters():
    seq_len = [7, 28, 35]
    learning_rate = [0.005, 0.01, 0.05]
    n_epochs = [100, 200, 300]
    n_batch = [50, 100, 500]
    hyp_parameters = list()
    for i in seq_len:
        for j in learning_rate:
            for k in n_epochs:
                for l in n_batch:
                    cfg = [i, j, k, l]
                    hyp_parameters.append(cfg)
    print('number of hyper parameters: %d' % len(hyp_parameters))
    return hyp_parameters

In [10]:
hyp_parameters = set_hyp_parameters()

number of hyper parameters: 81


# 예측모델 싱행

In [ ]:
# 0시부터 23시까지 예측 후 결합  
D = pd.DataFrame()

for i in range(0,24):
    df_Dh = df[(df['구분'] == 'D') & (df.index.hour == i)]
    df_Dh.drop(columns='구분', inplace=True)

    scaler = sklearn.preprocessing.MinMaxScaler(feature_range=(1, 2))
    df_Dh['공급량']=scaler.fit_transform(df_Dh['공급량'].values.reshape(-1,1))

    print('------------------------------구분: D, 시간: %d------------------------------' %i)
    
    seq_len, learning_rate, epochs, batch_size = search_parameter(df_Dh, hyp_parameters)

    X, y = load_data(df_Dh, seq_len)

    lstm_model = Sequential()

    lstm_model.add(LSTM(40,activation="tanh",return_sequences=True, input_shape=(X.shape[1],1)))

    lstm_model.add(LSTM(40,activation="tanh",return_sequences=True))

    lstm_model.add(LSTM(40,activation="tanh",return_sequences=False))

    lstm_model.add(Dense(1))
    
    adam = tf.keras.optimizers.Adam(lr=learning_rate)
    
    lstm_model.compile(optimizer=adam, loss=loss_nmae)
    model = lstm_model.fit(X, y, epochs=epochs, batch_size=batch_size, validation_split=0.2)

    test_Dh = test[(test['구분'] == 'D') & (test.index.hour == i)]
    test_Dh.drop(columns='구분', inplace=True)

    test_Dh['공급량'] = np.nan

    Dh = df_Dh.append(test_Dh)

    for j in range(len(test_Dh)):
        pred = lstm_model.predict(np.reshape(np.array(Dh.dropna()[len(Dh.dropna())-seq_len:]), (1, seq_len, 1)))
        Dh.iloc[len(Dh.dropna())] = pred

    Dh['공급량'] = scaler.inverse_transform(Dh)

    D = D.append(Dh)

D['구분'] = 'D'

------------------------------구분: D, 시간: 0------------------------------
------------------------------------------------------
Hyper Parameter: seq_len=7, learning_rate=0.01, epochs=100, batch_size=50
> 1) nmae: 0.02282
> 2) nmae: 0.03488
> 3) nmae: 0.02344
> 4) nmae: 0.02311
> 5) nmae: 0.02224
> 6) nmae: 0.02319
> 7) nmae: 0.02507
> 8) nmae: 0.02260
> 9) nmae: 0.02400
> 10) nmae: 0.02597
>> nmae mean: 0.02473
------------------------------------------------------
Hyper Parameter: seq_len=7, learning_rate=0.01, epochs=100, batch_size=100
> 1) nmae: 0.02318
> 2) nmae: 0.02278
> 3) nmae: 0.02361
> 4) nmae: 0.02272
> 5) nmae: 0.02282
> 6) nmae: 0.02247
> 7) nmae: 0.02482
> 8) nmae: 0.02984
> 9) nmae: 0.03487
> 10) nmae: 0.02283
>> nmae mean: 0.02499
------------------------------------------------------
Hyper Parameter: seq_len=7, learning_rate=0.01, epochs=100, batch_size=500
> 1) nmae: 0.03590
> 2) nmae: 0.04149
> 3) nmae: 0.04024
> 4) nmae: 0.04157
> 5) nmae: 0.03600
> 6) nmae: 0.0417

In [ ]:
# 생성된 D 확인
D

In [ ]:
# 시점으로 오름차순 정렬
D = D.sort_index()
D

In [ ]:
# 생성된 D 시각화
D.plot(figsize=(16,4),legend=True)

plt.title('구분: D, 시간별 공급량')

plt.show()

In [ ]:
# 생성된 D 예측부분 시각화
D['2019-1-1':'2019-3-31'].plot(figsize=(16,4),legend=True)

plt.title('구분: D, 시간별 공급량')

plt.show()